In [1]:
import os
import shutil
import random

BASE_PATH = "/kaggle/input/datasets/divyanshu007654321/fourth/train"
WORK_PATH = "/kaggle/working/vegetables"

# Create folders
for split in ["train", "val"]:
    os.makedirs(f"{WORK_PATH}/{split}/images", exist_ok=True)
    os.makedirs(f"{WORK_PATH}/{split}/labels", exist_ok=True)

# Get all images
images = [f for f in os.listdir(f"{BASE_PATH}/images") if f.endswith(('.jpg','.png','.jpeg'))]

random.shuffle(images)
split_idx = int(0.8 * len(images))  # 80% train, 20% val

train_images = images[:split_idx]
val_images = images[split_idx:]

def move_files(img_list, split):
    for img in img_list:
        label = img.rsplit(".", 1)[0] + ".txt"

        shutil.copy(f"{BASE_PATH}/images/{img}", f"{WORK_PATH}/{split}/images/{img}")
        shutil.copy(f"{BASE_PATH}/labels/{label}", f"{WORK_PATH}/{split}/labels/{label}")

move_files(train_images, "train")
move_files(val_images, "val")

print("✅ Dataset split completed")


✅ Dataset split completed


In [2]:
# %%writefile /kaggle/working/vegetables/data.yaml
# path: /kaggle/working/vegetables

# train: train/images
# val: val/images

# nc: 10
# names:
#   ['Capsicum', 'Lemon', 'Potato', 'carrot', 'chili pepper',
#    'coriander', 'eggplant', 'garlic', 'green peas', 'onion']


In [3]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install albumentations opencv-python tqdm

In [5]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# Paths
IMG_DIR = "/kaggle/working/vegetables/train/images"
LBL_DIR = "/kaggle/working/vegetables/train/labels"

OUT_IMG = "/kaggle/working/vegetables_aug/train/images"
OUT_LBL = "/kaggle/working/vegetables_aug/train/labels"

os.makedirs(OUT_IMG, exist_ok=True)
os.makedirs(OUT_LBL, exist_ok=True)

# Augmentation pipeline (REALISTIC)
transform = A.Compose(
    [
        A.RandomBrightnessContrast(p=0.6),
        A.HueSaturationValue(p=0.5),
        A.Rotate(limit=25, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomScale(scale_limit=0.3, p=0.5),
        A.MotionBlur(p=0.2),
        A.GaussianBlur(p=0.2),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.3
    )
)

AUG_PER_IMAGE = 5  # 🔥 230 → ~1150 images

for img_name in tqdm(os.listdir(IMG_DIR)):
    if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(IMG_DIR, img_name)
    lbl_path = os.path.join(LBL_DIR, img_name.rsplit(".", 1)[0] + ".txt")

    if not os.path.exists(lbl_path):
        continue

    image = cv2.imread(img_path)

    boxes = []
    labels = []

    with open(lbl_path, "r") as f:
        for line in f:
            cls, x, y, w, h = map(float, line.strip().split())
            boxes.append([x, y, w, h])
            labels.append(int(cls))

    for i in range(AUG_PER_IMAGE):
        augmented = transform(
            image=image,
            bboxes=boxes,
            class_labels=labels
        )

        aug_img = augmented["image"]
        aug_boxes = augmented["bboxes"]

        if len(aug_boxes) == 0:
            continue

        new_img_name = img_name.replace(".", f"_aug{i}.")
        new_lbl_name = new_img_name.rsplit(".", 1)[0] + ".txt"

        cv2.imwrite(os.path.join(OUT_IMG, new_img_name), aug_img)

        with open(os.path.join(OUT_LBL, new_lbl_name), "w") as f:
            for cls, box in zip(labels, aug_boxes):
                f.write(f"{cls} {' '.join(map(str, box))}\n")

print("✅ Offline augmentation completed")


100%|██████████| 208/208 [00:12<00:00, 16.89it/s]

✅ Offline augmentation completed


In [6]:
import os
import shutil

# Source folders
orig_img = "/kaggle/working/vegetables/train/images"
orig_lbl = "/kaggle/working/vegetables/train/labels"

aug_img = "/kaggle/working/vegetables_aug/train/images"
aug_lbl = "/kaggle/working/vegetables_aug/train/labels"

# Destination (merged)
dst_img = "/kaggle/working/vegetables_merged/train/images"
dst_lbl = "/kaggle/working/vegetables_merged/train/labels"

os.makedirs(dst_img, exist_ok=True)
os.makedirs(dst_lbl, exist_ok=True)

def copy_files(src, dst):
    for file in os.listdir(src):
        src_file = os.path.join(src, file)
        dst_file = os.path.join(dst, file)
        if not os.path.exists(dst_file):
            shutil.copy(src_file, dst_file)

# Copy original data
copy_files(orig_img, dst_img)
copy_files(orig_lbl, dst_lbl)

# Copy augmented data
copy_files(aug_img, dst_img)
copy_files(aug_lbl, dst_lbl)

print("✅ Original + Augmented data merged successfully")


✅ Original + Augmented data merged successfully


In [7]:
print("Images:", len(os.listdir(dst_img)))
print("Labels:", len(os.listdir(dst_lbl)))


Images: 1248
Labels: 1248


In [8]:
%%writefile /kaggle/working/vegetables_merged/data.yaml
path: /kaggle/working/vegetables_merged

train: train/images
val: /kaggle/working/vegetables/val/images

nc: 18
names:
  ['almond','apple', 'avocada', 'Broccoli', 'butter', 'Cinnamon','corn','egg','garlic','Lemon','mango','mushroom','onion','orange','pepper','Potato','rice','tomato']


Writing /kaggle/working/vegetables_merged/data.yaml


In [9]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # keep nano for small data

model.train(
    data="/kaggle/working/vegetables_merged/data.yaml",
    epochs=150,
    imgsz=640,
    batch=32,
    device=0,
    freeze=10,        # ⭐ VERY IMPORTANT
    augment=True,
    patience=30
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/vegetables_merged/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e00165f0980>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    

In [10]:
model = YOLO("runs/detect/train/weights/best.pt")
model.val(
    data="/kaggle/working/vegetables_merged/data.yaml"
)


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
Model summary (fused): 73 layers, 3,009,158 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1687.3±674.2 MB/s, size: 89.1 KB)
val: Scanning /kaggle/working/vegetables/val/labels.cache... 52 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 52/52 24.2Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.1it/s 1.9s0.7s
                   all         52        308      0.961      0.973      0.991      0.839
                almond         13         18      0.977          1      0.995      0.877
                 apple         10         12      0.958          1      0.995      0.905
               avocada         14         14      0.921          1      0.995      0.879
              Broccoli         14         21          1      0.972      0.995      0.852
                butter          9         5

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e02abbd55e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    

In [11]:
# from ultralytics import YOLO

# model = YOLO("/kaggle/working/runs/detect/veg4_yolo/yolov8_kaggle/weights/best.pt")

# metrics = model.val(
#     data="/kaggle/working/vegetables/data.yaml"
# )
